# Import Data, EDA

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

Take a quick look at the data we are working with

In [3]:
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [5]:
for i in range(50,60):
    print(train_data.text.iloc[i])
    print(train_data.target.iloc[i])

Deputies: Man shot before Brighton home set ablaze http://t.co/gWNRhMSO8k
1
Man wife get six years jail for setting ablaze niece
http://t.co/eV1ahOUCZA
1
SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintendent Lanford Salmon has r ... - http://t.co/vplR5Hka2u http://t.co/SxHW2TNNLf
0
Police: Arsonist Deliberately Set Black Church In North CarolinaåÊAblaze http://t.co/pcXarbH9An
1
Noches El-Bestia '@Alexis_Sanchez: happy to see my teammates and training hard ?? goodnight gunners.?????? http://t.co/uc4j4jHvGR'
0
#Kurds trampling on Turkmen flag later set it ablaze while others vandalized offices of Turkmen Front in #Diyala http://t.co/4IzFdYC3cg
1
TRUCK ABLAZE : R21. VOORTREKKER AVE. OUTSIDE OR TAMBO INTL. CARGO SECTION. http://t.co/8kscqKfKkF
1
Set our hearts ablaze and every city was a gift And every skyline was like a kiss upon the lips @Û_ https://t.co/cYoMPZ1A0Z
0
They sky was ablaze tonight in Los Angeles. I'm expecting IG and FB to be filled with sunset shots if I know my p

# Clean Up Tweets Before ML

Arguably the most important step (better data > better model) and after looking at some of the other entries, thinking there is plenty of room for improvement here.

TODO: Convert common shorthand (ie 'zzz') to words (ie 'Sleep')

In [6]:
from string import punctuation
import re
from nltk.corpus import stopwords
import nltk
import emoji
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text)
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

stopWords = set(stopwords.words('english'))

def clean_text(text):
    # change URL/@ tags to keywords
    text = ' '.join(['URL' if 'http://' in i or 'https://' in i else i for i in text.split(' ') ])
    text = ' '.join(['ATUSER' if len(i)>0 and i[0]=='@' else i for i in text.split(' ') ])
    #remove punctuation
    text = ''.join([c for c in text if c not in punctuation])
    #replace newline with space
    text = re.sub('\n',' ',text)
    #remove extra spaces
    text = re.sub(' +', ' ', text)
    #remove stop words
    text = ' '.join([i for i in text.lower().split(' ') if i not in stopWords])
    
    # some additional clean up steps I found interesting online
    text = re.sub(r'@[A-Za-z0-9_]+','',text)
    text = re.sub(r'RT : ','',text)
    text = re.sub(emoji.get_emoji_regexp(), r"", text)
    text = re.sub(r"<.*?>","",text)
    return text


def make_Lower(text):
    return str.lower(text)

train_data.text = train_data.text.apply(clean_text)
train_data.text = train_data.text.apply(correct_spellings)
text = train_data.text.tolist()

test_data.text = test_data.text.apply(clean_text)
test_data.text = test_data.text.apply(correct_spellings)
test_text = test_data.text.tolist()

# Tokenize Tweets

Create a vocabulary and use dictionaries to map words to integers/embeddings

In [7]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

all_words = flatten_list([i.split(' ') for i in text]) + flatten_list([i.split(' ') for i in test_text])

In [8]:
# feel free to use this import 
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(all_words)
vocab = sorted(counts,key=counts.get, reverse=True)
vocab_to_int = {word:i for i,word in enumerate(vocab,1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
tweet_ints = []
for tweet in text:
    tweet_ints.append([vocab_to_int[word] for word in tweet.split(' ')])
    
tweet_ints_test = []
for tweet in test_text:
    tweet_ints_test.append([vocab_to_int[word] for word in tweet.split(' ') if word in vocab_to_int.keys()])
    

tweet_ints_test = [i if len(i)>0 else [0] for i in tweet_ints_test]
tweet_ints = [i if len(i)>0 else [0] for i in tweet_ints]

In [9]:
#how big is vocab?
len(vocab)

19578

# Create Embedding Matrix

For ease, I used spaCy embeddings, but as potential improvement could experiment with other word embeddings like gloVe

In [10]:
import spacy
# Load the spacy model that you have installed
nlp = spacy.load('en_core_web_md')

#generate the embedding matrix
num_tokens = len(vocab)
embedding_dim = len(nlp('The').vector)
embedding_matrix = np.zeros((num_tokens+1, embedding_dim))
for i, word in enumerate(vocab):
    embedding_matrix[i+1] = nlp(word).vector

In [11]:
spacy_embedding_dim = embedding_matrix.shape

# Pad/Truncate Tokenized Tweets to Standard Length

In [12]:
# outlier review stats
tweet_lens = Counter([len(x) for x in tweet_ints])
print("Zero-length reviews: {}".format(tweet_lens[0]))
print("Maximum length: {}".format(max(tweet_lens)))

Zero-length reviews: 0
Maximum length: 25


In [13]:
# pad tweets to equal legnths
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

seq_length = 25

features = pad_features(tweet_ints, seq_length=seq_length)
sub_features = pad_features(tweet_ints_test, seq_length=seq_length)

# Create Train/Test Set, PyTorch Data Loaders

In [14]:
split_frac = 0.8

## split data into training, validation data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, test_x = features[:split_idx], features[split_idx:]
train_y, test_y = train_data.target.tolist()[:split_idx], train_data.target.tolist()[split_idx:]

print(train_x[0])
print(train_y[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0 5529  698  150   67 1912 3520   15]
1


In [27]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets, sub data is "submission data, running this data prep pipeline side by side with my training data"
train_data = TensorDataset(torch.from_numpy(np.array(train_x)), torch.from_numpy(np.array(train_y)))
test_data = TensorDataset(torch.from_numpy(np.array(test_x)), torch.from_numpy(np.array(test_y)))
sub_data = TensorDataset(torch.from_numpy(np.array(sub_features)), torch.from_numpy(np.zeros(len(sub_features))))

# set batch size to 32, may want to tinker with this
batch_size = 25

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,drop_last=True)
#Do NOT shuffle submission data
sub_loader = DataLoader(sub_data, shuffle=False, batch_size=1,drop_last=False)

In [16]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([25, 25])
Sample input: 
 tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0, 10687,   973,  3311,  3076, 10688,  3893,   200,
          1878,   391,  2243,   114,     1],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,  1840,    96,
           327,    59,   600, 11291,     1],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,    41,   952,     7,  2606,
            69,  2979,  1265,   533,     1],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,  1179,    73,
           884,    57,   368,   208,    89],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

# Define LSTM Model Class

I wanted to try something new with this kaggle competition.  I've used LSTM for sentiment analysis before and this type of binary classification seemed a similar enough task. I tweaked some of the parameters, but left the structure mostly the same with some notable exceptions. This was the first time I used bi-directional LSTM as well as a pre-trained word embedding for deep learning.  Fun Stuff!

In [17]:
train_on_gpu=torch.cuda.is_available()

In [18]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = spacy_embedding_dim[1]
hidden_dim = 256
n_layers = 2
embed_w =torch.tensor(embedding_matrix,dtype=torch.float32)

In [19]:
import rnn

params = {'vocab_size':vocab_size,'embed_dim':embedding_dim,'bidirectional':True,
          'rnn_hidden_dim':hidden_dim,'label_size':output_size,'linear_dims':[256],'rnn_type':2,
          'num_layers':2,'dropout':0.5,'attention_model':2}

print(params)

params = rnn.Parameters(params)

net = rnn.RnnClassifier('cuda',params,embed_w)
net.word_embeddings.requires_grad_(True)

{'vocab_size': 19579, 'embed_dim': 300, 'bidirectional': True, 'rnn_hidden_dim': 256, 'label_size': 1, 'linear_dims': [256], 'rnn_type': 2, 'num_layers': 2, 'dropout': 0.5, 'attention_model': 2}
Initialize layer with nn.init.xavier_uniform_: Linear(in_features=512, out_features=256, bias=True)
Initialize layer with nn.init.xavier_uniform_: Linear(in_features=256, out_features=1, bias=True)


C:\Users\duran\Desktop\Repos\disaster_tweets\rnn.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.word_embeddings.weight=nn.Parameter(torch.tensor(embedding_matrix,dtype=torch.float32))


Embedding(19579, 300)

# Train Model

In [20]:
# loss and optimization functions
import torch.nn as nn

lr=0.00002
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
PATH = 'mdl'

Save the best model based on validation accuracy

In [21]:
# training params

epochs = 10

counter = 0
print_every = 100
clip=5 # gradient clipping
best = 999

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        #net.hidden = tuple([each.data for each in net.hidden])
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output,h,attn = net(inputs,h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            
            val_losses = []
            net.eval()
            for inputs, labels in test_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output,val_h,attn = net(inputs,val_h)
                
                val_loss = criterion(output.squeeze(), labels.float())
                
                if val_loss < best:
                    torch.save(net,PATH)
                    best = val_loss

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            

Epoch: 1/10... Step: 100... Loss: 0.628473... Val Loss: 0.687031
Epoch: 1/10... Step: 200... Loss: 0.651022... Val Loss: 0.670263
Epoch: 2/10... Step: 300... Loss: 0.589234... Val Loss: 0.622819
Epoch: 2/10... Step: 400... Loss: 0.456959... Val Loss: 0.489640
Epoch: 3/10... Step: 500... Loss: 0.442715... Val Loss: 0.447912
Epoch: 3/10... Step: 600... Loss: 0.337966... Val Loss: 0.444856
Epoch: 3/10... Step: 700... Loss: 0.409808... Val Loss: 0.440891
Epoch: 4/10... Step: 800... Loss: 0.344096... Val Loss: 0.440665
Epoch: 4/10... Step: 900... Loss: 0.385145... Val Loss: 0.441650
Epoch: 5/10... Step: 1000... Loss: 0.404181... Val Loss: 0.437512
Epoch: 5/10... Step: 1100... Loss: 0.423316... Val Loss: 0.438095
Epoch: 5/10... Step: 1200... Loss: 0.466293... Val Loss: 0.452086
Epoch: 6/10... Step: 1300... Loss: 0.320618... Val Loss: 0.435431
Epoch: 6/10... Step: 1400... Loss: 0.482600... Val Loss: 0.441322
Epoch: 7/10... Step: 1500... Loss: 0.505603... Val Loss: 0.432488
Epoch: 7/10... Step

KeyboardInterrupt: 

In [22]:
# Get test data loss and accuracy

net = torch.load(PATH)
net.eval()

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    #h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h,attn = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.456
Test accuracy: 0.794


# Predict on the submission dataset

In [28]:
# Get test data loss and accuracy
all_preds = []
net = torch.load(PATH)
batch_size = 1

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data

for inputs, labels in sub_loader:
    
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h,attn = net(inputs, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()).item()  # rounds to the nearest integer
    all_preds.append(pred)



RuntimeError: invalid argument 7: equal number of batches expected at C:/w/1/s/tmp_conda_3.7_055457/conda/conda-bld/pytorch_1565416617654/work/aten/src\THC/generic/THCTensorMathBlas.cu:493

In [ ]:
test_data = pd.read_csv('test.csv')
test_data.shape

In [ ]:
test_data['target'] = all_preds

In [ ]:
test_data.filter(['id','target']).to_csv('sub.csv',index=False)